In [ ]:
import pandas as pd
import os

In [ ]:
def make_fasta(df_sco, dir_to_intergenic):
    
    df = pd.read_excel(df_sco, dtype=str, na_filter=False)
    
    #Intialize dataframe to be created
    names = []
    
    for i in df:
        names.append(i)
        
    df_final = pd.DataFrame(columns=names)
        
    os.chdir(dir_to_intergenic)
    
    for intergenic_file in os.listdir():
        if '.txt' in intergenic_file:
            with open(intergenic_file, 'r') as f:
                
                curr_name = None
                
                for i in df:
                    if str(i) in intergenic_file:
                        curr_name = str(i)
                
                headerList = []
                seqList = []
                currentSeq = 'placeholder' #place holder skips the first currentseq from being ''
                
                for line in f:
                    if line[0] == ">":
                        headerList.append(line[1:].strip())
                        if currentSeq != 'placeholder':
                            seqList.append(currentSeq)

                        currentSeq = ''

                    else:
                        currentSeq += line.strip()


                seqList.append(currentSeq)
                
                searchfor_list = []
                for value in df[curr_name]: #This will be specific for df column that contains gene IDs of interest
                    searchfor_list.append(value) 

                new_searchfor_list = []
                for item in searchfor_list: 
                    new_searchfor_list.append(item.split(','))

                combined_list = []
                for i in range(len(headerList)): 
                    combined_list.append(headerList[i] +'?'+seqList[i]) #This line can be mod to have seperator between id and seq, maybe '?'
                    #keep in mind, this will have > and \n for LF when written to txt file
                combined_list_Series = pd.Series(combined_list)
                    
    
                new_filter_list = []

                for value in new_searchfor_list:
                    working_list = []
                    for items in value: 
                        y = list(filter(lambda x:items in x, combined_list_Series))
                        if items == '':
                            working_list.append('')
                        elif y: 
                            working_list.append(y[0])

                    new_filter_list.append(working_list)
                    
            df_final[curr_name] = new_filter_list
            
    return(df_final)